电商18 180412126 王佳琦

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。
    #前面的（0.5，0.5，0.5） 是 R G B 三个通道上的均值， 后面(0.5, 0.5, 0.5)是三个通道的标准差，Normalize对每个通道执行以下操作：image =（图像-平均值）/ std

batch_size = 4

#batchsize: 批大小 num_works:num_works: 是否多进程读取数据 shuffle: 每个 epoch 是否乱序 drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [10]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [12]:
for epoch in range(8):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.225
[1,  4000] loss: 1.893
[1,  6000] loss: 1.713
[1,  8000] loss: 1.632
[1, 10000] loss: 1.555
[1, 12000] loss: 1.508
[2,  2000] loss: 1.414
[2,  4000] loss: 1.406
[2,  6000] loss: 1.371
[2,  8000] loss: 1.326
[2, 10000] loss: 1.343
[2, 12000] loss: 1.286
[3,  2000] loss: 1.220
[3,  4000] loss: 1.228
[3,  6000] loss: 1.204
[3,  8000] loss: 1.210
[3, 10000] loss: 1.183
[3, 12000] loss: 1.184
[4,  2000] loss: 1.098
[4,  4000] loss: 1.099
[4,  6000] loss: 1.116
[4,  8000] loss: 1.117
[4, 10000] loss: 1.115
[4, 12000] loss: 1.128
[5,  2000] loss: 1.033
[5,  4000] loss: 1.045
[5,  6000] loss: 1.026
[5,  8000] loss: 1.059
[5, 10000] loss: 1.056
[5, 12000] loss: 1.034
[6,  2000] loss: 0.966
[6,  4000] loss: 0.977
[6,  6000] loss: 0.986
[6,  8000] loss: 0.977
[6, 10000] loss: 1.000
[6, 12000] loss: 1.003
[7,  2000] loss: 0.904
[7,  4000] loss: 0.924
[7,  6000] loss: 0.944
[7,  8000] loss: 0.960
[7, 10000] loss: 0.956
[7, 12000] loss: 0.965
[8,  2000] loss: 0.852
[8,  4000] 

In [13]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)        #放进模型里
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()  #相同的累加

print('Accuracy of the network on the 2500 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 62 %


In [14]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}  #定义列表
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 63.7 %
Accuracy for class car   is: 71.7 %
Accuracy for class bird  is: 49.8 %
Accuracy for class cat   is: 36.2 %
Accuracy for class deer  is: 56.9 %
Accuracy for class dog   is: 64.5 %
Accuracy for class frog  is: 60.4 %
Accuracy for class horse is: 69.7 %
Accuracy for class ship  is: 73.8 %
Accuracy for class truck is: 74.6 %
